## Time series
Plik w którym tworze jeden odpowiedni szereg czasowy

In [1]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [5]:
tweets = pd.read_csv(f'../datasets/tweets_full.csv')

### Aggregation

In [49]:
# round down
def prepare_time(col):
    date, time = col.split(' ')
    time = f'{time[0:2]}:00:00'
    return date + ' ' + time

In [50]:
sentiment_val['date'] = sentiment_val['date'].apply(prepare_time)
sentiment_val['date'] = pd.to_datetime(sentiment_val['date'])
sentiment_val.head()

,pos,neu,neg,comp,date
0,0.122,0.878,0.000,0.2023,2015-01-01 01:00:00
1,0.000,0.629,0.371,-0.6486,2015-01-01 01:00:00
2,0.369,0.631,0.000,0.6249,2015-01-01 01:00:00
3,0.184,0.816,0.000,0.5423,2015-01-01 02:00:00
4,0.244,0.756,0.000,0.4404,2015-01-01 02:00:00


In [51]:
# grupuje godzinowo
test = pd.pivot_table(sentiment_val, index=['date'], aggfunc=['max', 'min', 'std', 'mean', 'median', 'count']).reset_index()

In [52]:
test.columns = [f'{i}_{j}' for i,j in test.columns]
test['count'] = test['count_pos']

In [53]:
test.drop(['count_neg','count_neu','count_pos', 'count_comp'], axis=1, inplace=True)

test.head()

,date_,max_comp,max_neg,max_neu,max_pos,min_comp,min_neg,min_neu,min_pos,std_comp,...,std_pos,mean_comp,mean_neg,mean_neu,mean_pos,median_comp,median_neg,median_neu,median_pos,count
0,2015-01-01 01:00:00,0.6249,0.371,1.000,0.369,-0.6486,0.0,0.629,0.000,0.530450,...,0.173949,0.044650,0.092750,0.784500,0.122750,0.10115,0.0000,0.7545,0.0610,4
1,2015-01-01 02:00:00,0.5423,0.223,1.000,0.244,-0.5766,0.0,0.756,0.000,0.432394,...,0.102317,0.187717,0.052333,0.825500,0.122000,0.30915,0.0000,0.8020,0.1520,6
2,2015-01-01 03:00:00,0.8065,0.279,1.000,0.327,-0.4767,0.0,0.673,0.000,0.494426,...,0.156752,0.176467,0.046500,0.852833,0.100667,0.00000,0.0000,0.8615,0.0000,6
3,2015-01-01 04:00:00,0.0000,0.231,1.000,0.000,-0.5106,0.0,0.769,0.000,0.361049,...,0.000000,-0.255300,0.115500,0.884500,0.000000,-0.25530,0.1155,0.8845,0.0000,2
4,2015-01-01 05:00:00,0.5994,0.000,0.868,0.565,0.4389,0.0,0.435,0.132,0.113491,...,0.306177,0.519150,0.000000,0.651500,0.348500,0.51915,0.0000,0.6515,0.3485,2


In [54]:
test = test.set_index('date_')
test_to_merge = test.between_time('00:00', '15:00')
test_to_merge.head()

,max_comp,max_neg,max_neu,max_pos,min_comp,min_neg,min_neu,min_pos,std_comp,std_neg,...,std_pos,mean_comp,mean_neg,mean_neu,mean_pos,median_comp,median_neg,median_neu,median_pos,count
date_,,,,,,,,,,,,,,,,,,,,,
2015-01-01 01:00:00,0.6249,0.371,1.000,0.369,-0.6486,0.0,0.629,0.000,0.530450,0.185500,...,0.173949,0.044650,0.092750,0.784500,0.122750,0.10115,0.0000,0.7545,0.0610,4
2015-01-01 02:00:00,0.5423,0.223,1.000,0.244,-0.5766,0.0,0.756,0.000,0.432394,0.091189,...,0.102317,0.187717,0.052333,0.825500,0.122000,0.30915,0.0000,0.8020,0.1520,6
2015-01-01 03:00:00,0.8065,0.279,1.000,0.327,-0.4767,0.0,0.673,0.000,0.494426,0.113901,...,0.156752,0.176467,0.046500,0.852833,0.100667,0.00000,0.0000,0.8615,0.0000,6
2015-01-01 04:00:00,0.0000,0.231,1.000,0.000,-0.5106,0.0,0.769,0.000,0.361049,0.163342,...,0.000000,-0.255300,0.115500,0.884500,0.000000,-0.25530,0.1155,0.8845,0.0000,2
2015-01-01 05:00:00,0.5994,0.000,0.868,0.565,0.4389,0.0,0.435,0.132,0.113491,0.000000,...,0.306177,0.519150,0.000000,0.651500,0.348500,0.51915,0.0000,0.6515,0.3485,2


In [55]:
test_to_merge = test_to_merge.groupby(pd.Grouper(freq='D'))
merged = test_to_merge.mean()
merged['count'] = test_to_merge['count'].sum()
merged.head()

,max_comp,max_neg,max_neu,max_pos,min_comp,min_neg,min_neu,min_pos,std_comp,std_neg,...,std_pos,mean_comp,mean_neg,mean_neu,mean_pos,median_comp,median_neg,median_neu,median_pos,count
date_,,,,,,,,,,,,,,,,,,,,,
2015-01-01,0.504653,0.087200,0.930267,0.245733,0.011033,0.000000,0.725133,0.069733,0.306077,0.054661,...,0.121633,0.263659,0.023872,0.835153,0.140963,0.263417,0.007700,0.836533,0.131167,49
2015-01-02,0.388713,0.101467,0.982133,0.182533,-0.197187,0.009067,0.780467,0.008800,0.239385,0.042339,...,0.078548,0.091726,0.030344,0.902989,0.066650,0.063907,0.012533,0.917267,0.035000,97
2015-01-03,0.591056,0.160562,0.938813,0.240062,-0.217550,0.000000,0.699438,0.033750,0.262513,0.049394,...,0.073858,0.421909,0.016536,0.813166,0.170297,0.543716,0.000000,0.796000,0.201594,185
2015-01-04,0.666394,0.059938,0.965750,0.309375,-0.071787,0.000000,0.690625,0.034250,0.232972,0.015764,...,0.085756,0.423062,0.004947,0.827971,0.167083,0.514200,0.000000,0.808750,0.191250,217
2015-01-05,0.835862,0.088750,0.991750,0.371188,-0.194500,0.000000,0.628812,0.000000,0.303786,0.025652,...,0.113254,0.406652,0.011029,0.819593,0.169376,0.422047,0.000000,0.817188,0.156562,288


In [56]:
merged = merged.reset_index()
merged.date_ = merged['date_'] +  pd.to_timedelta(1, unit='H')
merged = merged.set_index('date_')
merged.head()

,max_comp,max_neg,max_neu,max_pos,min_comp,min_neg,min_neu,min_pos,std_comp,std_neg,...,std_pos,mean_comp,mean_neg,mean_neu,mean_pos,median_comp,median_neg,median_neu,median_pos,count
date_,,,,,,,,,,,,,,,,,,,,,
2015-01-01 01:00:00,0.504653,0.087200,0.930267,0.245733,0.011033,0.000000,0.725133,0.069733,0.306077,0.054661,...,0.121633,0.263659,0.023872,0.835153,0.140963,0.263417,0.007700,0.836533,0.131167,49
2015-01-02 01:00:00,0.388713,0.101467,0.982133,0.182533,-0.197187,0.009067,0.780467,0.008800,0.239385,0.042339,...,0.078548,0.091726,0.030344,0.902989,0.066650,0.063907,0.012533,0.917267,0.035000,97
2015-01-03 01:00:00,0.591056,0.160562,0.938813,0.240062,-0.217550,0.000000,0.699438,0.033750,0.262513,0.049394,...,0.073858,0.421909,0.016536,0.813166,0.170297,0.543716,0.000000,0.796000,0.201594,185
2015-01-04 01:00:00,0.666394,0.059938,0.965750,0.309375,-0.071787,0.000000,0.690625,0.034250,0.232972,0.015764,...,0.085756,0.423062,0.004947,0.827971,0.167083,0.514200,0.000000,0.808750,0.191250,217
2015-01-05 01:00:00,0.835862,0.088750,0.991750,0.371188,-0.194500,0.000000,0.628812,0.000000,0.303786,0.025652,...,0.113254,0.406652,0.011029,0.819593,0.169376,0.422047,0.000000,0.817188,0.156562,288


In [57]:
# drop merged
print(len(test))
test = test.drop(index=test.between_time('00:00', '15:00').index)
print(len(test))

43055
14514


In [58]:
connected = pd.concat([test, merged], axis=0)
len(connected)

16341

In [59]:
connected = connected.sort_index()
connected.head(10)

,max_comp,max_neg,max_neu,max_pos,min_comp,min_neg,min_neu,min_pos,std_comp,std_neg,...,std_pos,mean_comp,mean_neg,mean_neu,mean_pos,median_comp,median_neg,median_neu,median_pos,count
date_,,,,,,,,,,,,,,,,,,,,,
2015-01-01 01:00:00,0.504653,0.087200,0.930267,0.245733,0.011033,0.000000,0.725133,0.069733,0.306077,0.054661,...,0.121633,0.263659,0.023872,0.835153,0.140963,0.263417,0.007700,0.836533,0.131167,49
2015-01-01 16:00:00,0.750600,0.000000,1.000000,0.330000,0.000000,0.000000,0.670000,0.000000,0.375300,0.000000,...,0.165000,0.187650,0.000000,0.917500,0.082500,0.000000,0.000000,1.000000,0.000000,4
2015-01-01 17:00:00,0.571900,0.126000,1.000000,0.323000,0.000000,0.000000,0.677000,0.000000,0.236067,0.054317,...,0.114193,0.243800,0.030000,0.831833,0.138167,0.177700,0.000000,0.783500,0.185500,12
2015-01-01 18:00:00,0.636900,0.136000,1.000000,0.292000,-0.296000,0.000000,0.638000,0.000000,0.302952,0.040021,...,0.127413,0.174121,0.015000,0.896071,0.088857,0.000000,0.000000,1.000000,0.000000,14
2015-01-01 19:00:00,0.440400,0.207000,1.000000,0.289000,-0.526700,0.000000,0.711000,0.000000,0.340974,0.069000,...,0.109983,0.231278,0.023000,0.850778,0.126222,0.431000,0.000000,0.868000,0.132000,9
2015-01-01 20:00:00,0.340000,0.000000,0.876000,0.124000,0.340000,0.000000,0.876000,0.124000,NaN,NaN,...,NaN,0.340000,0.000000,0.876000,0.124000,0.340000,0.000000,0.876000,0.124000,1
2015-01-01 21:00:00,0.821700,0.000000,1.000000,0.322000,0.000000,0.000000,0.678000,0.000000,0.581030,0.000000,...,0.227688,0.410850,0.000000,0.839000,0.161000,0.410850,0.000000,0.839000,0.161000,2
2015-01-01 22:00:00,0.802000,0.153000,1.000000,0.444000,-0.440400,0.000000,0.556000,0.000000,0.486241,0.068153,...,0.195397,0.174440,0.045400,0.828000,0.126600,0.000000,0.000000,0.847000,0.000000,5
2015-01-01 23:00:00,0.658800,0.000000,0.899000,0.196000,0.202300,0.000000,0.804000,0.101000,0.322794,0.000000,...,0.067175,0.430550,0.000000,0.851500,0.148500,0.430550,0.000000,0.851500,0.148500,2


In [60]:
stocks = pd.read_csv(f'stocks_clean/{company_stock}.csv', parse_dates=['date_'])
#stocks_1 = pd.read_csv('stocks_clean/GOOG.csv', parse_dates=['date_'])
#stocks_2 = pd.read_csv('stocks_clean/GOOGL.csv', parse_dates=['date_'])
#stocks = pd.concat([stocks_1, stocks_2], axis=0)
stocks = stocks.set_index('date_')
stocks.sample(20)

,ticker,open,high,low,close,vol
date_,,,,,,
2018-06-04 20:00:00,GOOG,1135.560,1138.1800,1135.1200,1138.1800,52470
2017-07-20 16:00:00,GOOG,974.420,975.9000,973.1100,974.9050,122695
2017-03-06 18:00:00,GOOG,824.975,825.7000,823.5042,824.6200,95842
2015-04-10 16:00:00,GOOGL,549.520,549.8500,545.3900,546.0400,135162
2017-01-30 22:00:00,GOOG,802.300,803.1800,801.0000,802.6950,255324
2017-01-27 19:00:00,GOOGL,853.695,854.4400,848.4150,851.4100,225318
2018-03-05 20:00:00,GOOGL,1092.860,1095.8000,1090.1400,1094.2800,56968
2020-07-27 21:00:00,GOOGL,1527.000,1531.9700,1525.7800,1526.0000,2100
2020-05-18 22:00:00,GOOG,1387.100,1391.0000,1380.1400,1384.0900,11838


In [61]:
test = stocks.merge(connected, how='outer', left_index=True, right_index=True)

In [62]:
test = test.dropna()

In [63]:
test.to_csv(f'stocks_sentiment/{company_name}.csv')

In [64]:
full_stocks = pd.DataFrame()
for c_name in ['amazon.com', 'apple', 'google', 'microsoft', 'tesla']:
    part = pd.read_csv(f'stocks_sentiment/{c_name}.csv', parse_dates=['date_'])
    full_stocks = pd.concat([full_stocks, part], axis=0)

full_stocks.to_csv('full_clean.csv', index=False)